In [1]:

import sys
import os
import pandas as pd
import numpy as np
import importlib
propject_root = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..'))
sys.path.append(propject_root)


import src.database.redshift_connection as redshift_connection
import src.plotly_wrapper.scatter as sc_wrapper
import src.plotly_wrapper.box as box_wrapper
import src.plotly_wrapper.bar as bar_wrapper
import src.plotly_wrapper.figure as fig_wrapper
import src.plotly_wrapper.trace as tc
import src.utils.calendar_utils as cu


In [ ]:
conn = redshift_connection.RedshiftConnection()

In [ ]:
def get_results(read_from_file=False, conn=None):
    if read_from_file:
        results = pd.read_csv('/Users/ahmed.sobih/recon_results.csv').drop_duplicates()
    else:
        f = open('../sql/queries/raw_records.sql', 'r')
        try:
            query = f.read()
        finally:
            f.close()
        results = conn.execute_query(query)
    return results
# results = get_results(read_from_file=False, conn=conn)

results = get_results(read_from_file=True, conn=None)

# Data Exploration

## Records per city count

In [ ]:
def plot_records_per_city(results):
    importlib.reload(bar_wrapper)
    importlib.reload(fig_wrapper)

    num_records_per_city = results.groupby('city').agg(
        num_records = ('remittance', 'size')
    ).reset_index().sort_values(by='num_records', ascending=False)
    num_records_per_city['num_records'] = num_records_per_city['num_records'].astype(int)
    num_records_per_city['percent_of_total_text'] = np.round(num_records_per_city['num_records']/len(results)*100, 1).astype(str) + '%'
    num_records_per_city['num_records_text'] = np.round(num_records_per_city['num_records']/1000000, 2).astype(str) + 'M'
    bar = bar_wrapper.Bar(
        df=num_records_per_city,
        x_col='city',
        y_col='num_records',
        name='', 
        text=list(num_records_per_city.percent_of_total_text)
    )
    fig = fig_wrapper.Figure([bar], None, 'Number of records per city')
    fig.set_axis_title(xaxis_title='City', yaxis_title='Number of records')
    fig.show_legend=False
    fig.show()
    fig.save('records_per_city.jpg')

plot_records_per_city(results)

## Zero and NULL value

In [ ]:
def get_remittance_net_earnings_stats(results):
    rec_per_city = results.groupby('city').agg(
        num_records = ('remittance', 'size'),  
        percent_of_total = ('remittance', lambda x: np.round(len(x)/len(results), 2)),
        zero_remittance_percentage = ('remittance', lambda x: np.round(len(x[x==0])/len(x), 2)),
        null_remittance_percentage = ('remittance', lambda x: np.round(len(x[x.isnull()])/len(x), 2)), 
        zero_net_earning_percentage = ('net_earning', lambda x: np.round(len(x[x==0])/len(x), 2)),
        null_net_earning_percentage = ('net_earning', lambda x: np.round(len(x[x.isnull()])/len(x), 2)),
    ).reset_index().sort_values(by='zero_remittance_percentage', ascending=False)

    rec_per_city['zero_remittance_percentage'] = rec_per_city['zero_remittance_percentage'].astype(float)
    rec_per_city['null_remittance_percentage'] = rec_per_city['null_remittance_percentage'].astype(float)
    rec_per_city['num_records'] = rec_per_city['num_records'].astype(int)
    rec_per_city.sort_values(by='num_records', ascending=False)
    return rec_per_city
get_remittance_net_earnings_stats(results)


In [ ]:
print('Percenate of zero remittance: ' , np.round(len(results[(results.remittance==0)])/len(results), 2)*100)
print('Percenate of zero net_earning: ' , np.round(len(results[(results.net_earning==0)])/len(results), 2)*100)


## Remittance and Net earnings Percentiles

In [ ]:
## Create a dataframe with the quantiles of the remittance and net_earning for each city
def create_quantiles_df(results, col='remittance'):
    quantiles = results.groupby('city')[col].quantile(np.arange(0, 1.01, 0.01)).reset_index()
    quantiles.rename(columns={'level_1': 'quantile', col: f'{col}_value'}, inplace=True)
    pivot_quantiles = quantiles.pivot(
        index='quantile', 
        columns='city', 
        values=f'{col}_value'
    ).reset_index()
    pivot_quantiles = pivot_quantiles[['quantile', 'Lagos', 'Accra', 'Johannesburg', 'Cape Town', 
                            'Nairobi', 'Ibadan', 'mumbai', 'bangalore', 'hyderabad']]
    pivot_quantiles.to_csv(f'../data/processed/{col}_quantiles.csv', index=False)
    
    return pivot_quantiles
create_quantiles_df(results, col='remittance')
create_quantiles_df(results, col='net_earning')

In [ ]:
def plot_value_range(results, col='remittance'):
    importlib.reload(fig_wrapper)
    importlib.reload(box_wrapper)

    results_with_quantiles = results.merge(
        results.groupby('city')[col].quantile(0.98).reset_index().rename(
            columns={col: f'{col}_98'}
        ), on='city', how='left'
    ).merge(
        results.groupby('city')[col].quantile(0.01).reset_index().rename(
            columns={col: f'{col}_01'}
        ), on='city', how='left'
    )
    plot_data = results_with_quantiles[
                (results_with_quantiles[col] <= results_with_quantiles[f'{col}_98']) & 
                (results_with_quantiles[col] >= results_with_quantiles[f'{col}_01']) & 
                (results_with_quantiles[col] != 0)
            ]
    box = box_wrapper.Box(
        df= plot_data, 
        x_col='city',
        y_col=col,
        legendgrouptitle='City',
        name='', boxpoints=False
    )

    title = f'{col.replace("_", " ").capitalize()} Value Range'
    
    fig = fig_wrapper.Figure([box], None, title)

    for city in results_with_quantiles.city.unique():
        median_value = plot_data[plot_data.city==city][col].median()
        y = plot_data[plot_data.city==city][col].max()
        fig.add_annotation(
            x=city,
            y=y+1500,
            text=f'{np.round(median_value/1000, 1)}k',
            showarrow=False
        )
    fig.set_axis_title(xaxis_title='City', yaxis_title=title)
    fig.show()
    fig.show_legend=False
    fig.save(f'{col}_value_range.jpg')

plot_value_range(results, col='remittance')
plot_value_range(results, col='net_earning')

## Weekly Insights
### Create Weekly data

In [9]:
def get_filtered_results(results, filter_zero_remittance=True, filter_zero_net_earning=True):
    filtered_results = results[
        (~results.drn.isnull()) &
        (~results.city.isnull()) & 
        (~results.week.isnull()) & 
        (results.week <= '2025-03-10')
    ].drop_duplicates()
    if filter_zero_remittance:
        filtered_results = filtered_results[filtered_results.remittance > 0]
    if filter_zero_net_earning:
        filtered_results = filtered_results[filtered_results.net_earning > 0]
    col = 'remittance'
    filtered_results = filtered_results.merge(
            results.groupby('city')[col].quantile(0.98).reset_index().rename(
                columns={col: f'{col}_98'}
            ), on='city', how='left'
        ).merge(
            results.groupby('city')[col].quantile(0.01).reset_index().rename(
                columns={col: f'{col}_01'}
            ), on='city', how='left'
        )
    
    return filtered_results

def get_weekly_results(filtered_results):
    filtered_results = get_filtered_results(results)
    weekly_agg = filtered_results.groupby(['city', 'drn', 'week']).agg(
        remittance = ('remittance', 'sum'),
        net_earning = ('net_earning', 'sum'),
        num_records = ('remittance', 'size'),
    ).reset_index()
    
    weekly_agg['week'] = pd.to_datetime(weekly_agg['week'])
    
    weekly_agg['week_num']= weekly_agg.sort_values(
        ['city', 'drn', 'week'], ascending=True
    ).groupby(['city', 'drn']).cumcount()

    weekly_agg['is_remittance_paid'] = np.where(
        weekly_agg.remittance <= weekly_agg.net_earning, 
        1, 
        0
    )
    first_week = weekly_agg.groupby(['city', 'drn']).agg(
        first_week = ('week', 'min'),
    ).reset_index()
    weekly_agg = weekly_agg.merge(first_week, on=['city', 'drn'], how='left')
    weekly_agg['first_month'] = weekly_agg.first_week.dt.to_period('M').dt.end_time

    #weekly_agg['week_num'] = weekly_agg.sort_values(['city', 'drn', 'week']).groupby(['city', 'drn']).cumcount()
    return weekly_agg

weekly_agg = get_weekly_results(results)


In [ ]:
weekly_agg.dtypes

In [11]:
def get_weeks_calendar(weekly_agg):
    weeks = weekly_agg[['week']].drop_duplicates()

    importlib.reload(cu)

    calendar = cu.create_week_calendar(
        start_date='2020-01-01',
        end_date=weekly_agg.week.max().strftime('%Y-%m-%d')
    )
    weeks = weeks.merge(
        calendar, left_on='week', 
        right_on = 'week_start', 
        how='left'
    ).sort_values(by='week_start')
    
    wrong_week_tag = weekly_agg[
        weekly_agg.week.isin(
            weeks[weeks.week_start.isnull()].week
        )
    ].groupby(['city', 'week']).agg(
        num_records = ('remittance', 'size'),
    ).reset_index()
    wrong_week_tag.to_csv('../data/processed/wrong_week_tag.csv', index=False)
    return weeks[~weeks.week_start.isnull()]


def get_city_week_results(weekly_agg, groupby_columns=['city', 'week']):
    city_week_results = weekly_agg.groupby(groupby_columns).agg(
        num_records = ('remittance', 'size'),
        remittance = ('is_remittance_paid', 'sum'),
    ).reset_index()
    city_week_results['is_remittance_paid_percentage'] = city_week_results['remittance']/city_week_results['num_records']
    
    weeks = get_weeks_calendar(weekly_agg)
    if 'city' in groupby_columns:
    # Create a dataframe is a product of weeks and cities
        import itertools
        weeks_cities = pd.DataFrame(
            list(itertools.product(weeks.week, weekly_agg.city.unique())),
            columns=['week', 'city'] 
        )
    else:
        weeks_cities = weeks
    city_week_results = weeks_cities.merge(
        city_week_results,
        on=groupby_columns,
        how='left'
    ).sort_values(by=groupby_columns, ascending=True)
    city_week_results['num_records'] = city_week_results['num_records'].fillna(0)
    city_week_results['remittance'] = city_week_results['remittance'].fillna(0)
    if 'city' in groupby_columns:
        city_start_date = city_week_results[city_week_results.num_records>0].groupby('city').agg(
            start_date = ('week', 'min')
        ).reset_index()
        city_week_results = city_week_results.merge(
            city_start_date,
            on='city',
            how='left'
        )   
        city_week_results = city_week_results[city_week_results.week>=city_week_results.start_date]
    city_week_results['is_remittance_paid_percentage'] = city_week_results['is_remittance_paid_percentage'].fillna(0)
    return city_week_results

city_week_results = get_city_week_results(weekly_agg)
all_results = get_city_week_results(weekly_agg, groupby_columns=['week'])

In [ ]:
def plot_weekly_stats(results, y_col='num_records'):
    importlib.reload(fig_wrapper)
    importlib.reload(sc_wrapper)
    traces= []
    
    file_name = f'weekly_{y_col}'
    if y_col == 'num_records':
        title = f'Weekly # of remittance records'
        yaxis_title= '# of records'
    elif y_col == 'is_remittance_paid_percentage':
        title = f'Weekly % of remittance fully covered'
        yaxis_title= '% of remittance fully covered'
    else:
        title = f'Weekly {y_col.replace("_", " ")}'
        yaxis_title= y_col.replace("_", " ").capitalize()
        
    if 'city' in results.columns:
        for city in results.city.unique(): 
            scatter = sc_wrapper.Scatter(
                df= results[results.city==city], 
                x_col='week',
                y_col=y_col,
                mode='lines',
                legendgrouptitle='City',
                name=city
            )
            traces.append(scatter)
        file_name = f'city_{file_name}'
        title = f'{title} per city'  
    else:
        scatter = sc_wrapper.Scatter(
            df= results, 
            x_col='week',
            y_col=y_col,
            mode='lines',
            name=''
        )
        traces.append(scatter)

    
    fig = fig_wrapper.Figure(traces, None, title)
    fig.set_axis_title(xaxis_title='Week', yaxis_title=yaxis_title)
    if not 'city' in results.columns:
        fig.show_legend=False
    if y_col=='is_remittance_paid_percentage':
        fig.set_percentage_axis()
    fig.show()
    fig.save(file_name + '.jpg')

plot_weekly_stats(results = city_week_results, y_col='num_records')
plot_weekly_stats(results = all_results, y_col='num_records')

In [ ]:
plot_weekly_stats(results = city_week_results, y_col='is_remittance_paid_percentage')
plot_weekly_stats(results = all_results, y_col='is_remittance_paid_percentage')

## Cohort Analysis

In [14]:
def get_drn_stats(results_agg):
    results_agg_copy = results_agg.copy(deep=True)
   
    weeks = get_weeks_calendar(results_agg)
    import itertools
    city_drn = results_agg[['city', 'drn']].drop_duplicates()
    city_drn['tmp_col'] = 1
    weeks['tmp_col'] = 1
    weeks_city_drn = city_drn.merge(weeks, on='tmp_col', how='left')
    del city_drn['tmp_col']
    # weeks_city_drn = pd.DataFrame(
    #     list(itertools.product(weeks.week, weekly_agg.drn.unique())),
    #     columns=['week', 'drn'] 
    # )
    drn_stats = weeks_city_drn.merge(
        results_agg_copy, 
        on=['city', 'week', 'drn'], 
        how='left'
    )
    
    drn_start_date = drn_stats[drn_stats.num_records>0].groupby(['city', 'drn']).agg(
        start_date = ('week', 'min')
    ).reset_index()
    drn_stats = drn_stats.merge(drn_start_date, on=['city', 'drn'], how='left')
    drn_stats = drn_stats[drn_stats.week>=drn_stats.start_date]
    drn_stats['week_num'] = drn_stats.sort_values(
        ['city', 'drn', 'week'], ascending=True
    ).groupby(['city', 'drn']).cumcount()
    
    drn_stats['month'] = drn_stats.week.dt.to_period('M')
    return drn_stats
drn_stats = get_drn_stats(weekly_agg)


In [ ]:
def get_drn_month_cohorts(drn_stats):
    drn_month_cohorts = drn_stats.groupby(['first_month']).agg(
        num_drns = ('drn', 'nunique')
    ).reset_index()
    return drn_month_cohorts

def plot_drn_month_cohorts(drn_stats):
    importlib.reload(fig_wrapper)
    importlib.reload(sc_wrapper)
    drn_month_cohorts = get_drn_month_cohorts(drn_stats)
    traces= []
    scatter = sc_wrapper.Scatter(
            df= drn_month_cohorts, 
            x_col='first_month',
            y_col='num_drns',
            mode='lines',
            legendgrouptitle='City',
            name=f''
        )
    traces.append(scatter)
    fig = fig_wrapper.Figure(traces, None, title=f'Cohort size overtime')
    fig.set_axis_title(xaxis_title='Month', yaxis_title='Cohort size')
    fig.show_legend=False
    fig.show()
    fig.save('drn_month_cohorts.jpg')
plot_drn_month_cohorts(drn_stats)

In [ ]:
def plot_cohort_heatmap(drn_stats):
    drn_month_cohorts= get_drn_month_cohorts(drn_stats)
    drn_month_cohorts = drn_month_cohorts.merge(
        drn_stats.groupby(['first_month', 'week']).agg(
            paid_remittance = ('is_remittance_paid', 'sum'),
        ).reset_index(),
        on=['first_month'],
        how='left'
    ).sort_values(by=['first_month', 'week'], ascending=True)
    drn_month_cohorts['is_remittance_paid_percentage'] = drn_month_cohorts['paid_remittance']/drn_month_cohorts['num_drns']


    cohort_pivot= drn_month_cohorts.pivot(
        index='first_month', 
        columns='week', 
        values='is_remittance_paid_percentage'
    )

    import plotly.graph_objects as go


    fig = go.Figure(
        data=go.Heatmap(
        z=cohort_pivot,
        x=cohort_pivot.columns,
        y=cohort_pivot.index,
        colorscale='YlGn',  # Built-in color scale
    ))
    default_layout = fig_wrapper.Figure([], None, 'Cohort % of remittance paid')._default_layout()
    default_layout['xaxis']['title']['text'] = 'Week #'
    default_layout['yaxis']['title']['text'] = 'Cohort Month'
    default_layout['yaxis']['autorange'] = 'reversed'
    fig.layout= default_layout
    fig.show()
    fig.write_image(
        "../reports/plots/" + 'cohort_analysis_heatmap.jpg', 
        format="jpeg",
        width=600, 
        height=400, 
        scale=6,
        engine="kaleido"
    )

plot_cohort_heatmap(drn_stats)

# Vitalik Query Results

In [ ]:
def run_vitlik_data(file_name='../data/processed/vitalik_query_results.csv'):
    importlib.reload(fig_wrapper)
    importlib.reload(sc_wrapper)
    if file_name is not None:
        test_results = pd.read_csv(file_name)
    else:
        f = open('../sql/queries/vitalik_query.sql', 'r')
        try:
            query = f.read()
        finally:
            f.close()
        test_results = conn.execute_query(query)
        test_results.to_csv('../data/processed/vitalik_query_results.csv', index=False)
    traces=[]
    for country in test_results.country.unique():
        scatter = sc_wrapper.Scatter(
                    df= test_results[test_results.country==country], 
                    x_col='week',
                    y_col='pct_drivers_earning_more_than_remittance',
                    mode='lines',
                    legendgrouptitle='City',
                    name=country
                )
        traces.append(scatter)
    fig = fig_wrapper.Figure(traces, None, title='test')
    fig.set_axis_title(xaxis_title='Week', yaxis_title='yaxis_title')
    fig.show()
        
run_vitlik_data()